## Split dataset

Testing dataset: Cam 2 at 220430 220508 & Cam 4 at 220502 & 220510

In [ ]:
import os
import numpy as np
import pandas as pd
import json 

In [ ]:
root = os.getcwd() + '/../../data/birds/annotations'
pathin = root + '/coco_test_dataset.json'
date_list = ['220430', '220508', '220502','220510']

with open(pathin, "r") as f:
    data = json.load(f)

In [ ]:
filenames = [i['file_name'].split('/')[-1] for i in data['images']]
img_id = [i['id'] for i in data['images']]
dates = [f.split('_')[-2] for f in filenames]
cond = [d in date_list for d in dates]

## retrieve image id
test_img_id = np.array(img_id)[cond]
train_img_id = np.array(img_id)[np.invert(cond)]

def get_new_dataset(data, test_img_id):

    img_list = [d for d in data['images'] if d['id'] in test_img_id]
    anno_list = [d for d in data['annotations'] if d['image_id'] in test_img_id]

    data_test = {}
    data_test['images'] = img_list
    data_test['annotations'] = anno_list
    data_test['categories'] = data['categories']

    return data_test

## create new datasets
data_test = get_new_dataset(data, test_img_id)
data_train = get_new_dataset(data, train_img_id)

In [ ]:
len(data_test['images'])

In [ ]:
## save new datasets
with open(root + '/test_empty_220510.json', "w") as f:
    json_string = json.dumps(data_test)
    f.write(json_string)

In [ ]:
with open(root + '/coco_valtrain_dataset.json', "w") as f:
    json_string = json.dumps(data_train)
    f.write(json_string)

Split Train/Validation

In [ ]:
with open(root + '/coco_train_dataset.json', "r") as f:
    data_train = json.load(f)

In [ ]:
num_val = int(len(data['images'])*0.1)
train_id = [d['id'] for d in data_train['images']]
np.random.shuffle(train_id)
val_id = train_id[:num_val]
id_left = train_id[num_val:]

In [ ]:
cond = [d in val_id for d in train_id]

## create new image list
img_val = list(np.array(data_train['images'])[cond])
img_train = list(np.array(data_train['images'])[np.invert(cond)])

## create new annotations list
anno_val = [a for a in data_train['annotations'] if a['image_id'] in val_id]
anno_train = [a for a in data_train['annotations'] if a['image_id'] in id_left]

In [ ]:
new_data_val = {}
new_data_val['images'] = img_val
new_data_val['annotations'] = anno_val
new_data_val['categories'] = data_train['categories']

new_data_train = {}
new_data_train['images'] = img_train
new_data_train['annotations'] = anno_train
new_data_train['categories'] = data_train['categories']

In [ ]:
with open(root + '/coco_train_dataset.json', "w") as f:
    json_string = json.dumps(new_data_train)
    f.write(json_string)

In [ ]:
with open(root + '/coco_val_dataset.json', "w") as f:
    json_string = json.dumps(new_data_val)
    f.write(json_string)

In [ ]:
len(new_data_train['images']) + len(new_data_val['images']), len(data_train['images'])

## Create sub files

In [ ]:
ds_train = 'full_coco_train.json'
ds_val = 'full_coco_val.json'
ds_test = 'full_coco_test.json'


In [ ]:
def get_sample(ds_name, N):

    root = os.getcwd() + '/../data/birds/annotations/'

    with open(root + ds_name, "r") as f:
        data = json.load(f)
    ## get images IDs
    img_id = [d['id'] for d in data['images']]
    np.random.shuffle(img_id)
    ## get the new list
    img_list = [i for i in data['images'] if i['id'] in img_id[:N]]
    anno_list = [a for a in data['annotations'] if a['image_id'] in img_id[:N]]

    new_file = {}
    new_file['images'] = img_list
    new_file['annotations'] = anno_list
    new_file['categories'] = data['categories']

    with open(root + 'sub_' + ds_name, "w") as f:
        json_string = json.dumps(new_file)
        f.write(json_string)



In [ ]:
get_sample(ds_val, 10)